In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [4]:
!pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Imports

In [5]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from dataloader import BalancedMelSampler, MelSampler, convert_csv_to_dict_for_dataloader
from losses import NpairsLoss
from metrics import TFLWLRAP
from split_data import get_split
from train import get_model, get_callbacks, get_lr_metric
from models import NUM_FRAMES, Classifier, DeepMetricLearning

In [6]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [8]:
def get_lr(epoch, lr, min_lr=1e-5, max_lr=1e-3, epochs=30, warmup_prop=0.1):   
    if epoch <= epochs * warmup_prop:
        return min_lr + max_lr - (max_lr * (epochs * warmup_prop - epoch) / (epochs * warmup_prop))
    else:
        return min_lr + max_lr - (max_lr * (epoch - epochs * warmup_prop) / (epochs - epochs * warmup_prop))

In [9]:
def main(fold_idx, saved_path, pretrained_path, pretrained_with_contrastive):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)
    pretrained_with_contrastive = bool(pretrained_with_contrastive)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")
    train_index, val_index = get_split(fold=fold_idx)
    fold_train_dict = convert_csv_to_dict_for_dataloader(train_data.iloc[train_index])
    fold_valid_dict = convert_csv_to_dict_for_dataloader(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=32,
        max_length=NUM_FRAMES,
        is_train=True,
        n_classes=24,
        use_cutmix=True,
        cache=True,
        n_classes_in_batch=8,
        shuffle_aug=False,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=balanced_train_data_loader.batch_size,
        n_classes=balanced_train_data_loader.n_classes,
        cache=True,
        max_length=NUM_FRAMES,
        is_train=False,
        use_cutmix=False,
        shuffle_aug=balanced_train_data_loader.shuffle_aug,
    )
    
#     return 0

    print(' -> Preparing Model \n')
    model = get_model(
        saved_path=saved_path,
        pretrained_with_contrastive=pretrained_with_contrastive,
        pretrained_path=pretrained_path,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=1e-4,
        maximal_learning_rate=1e-3,
        step_size=15 * 20,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap")],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    )   

    print(' -> Training Model \n')

    callbacks = get_callbacks(pretrained_with_contrastive, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=60,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [10]:
DEBUG = False
pretrained_folder = "../logs/2021-01-28/1/"   # simple
pretrained_folder = "../logs/2021-01-30/42/"  # mixstyle + iunterpolate + cbam
pretrained_folder = "../logs/2021-01-31/8/"  # mixstyle + iunterpolate + cbam without relu
log_folder = "../logs/"

In [11]:
backbones = [
    "efficientnetb0",
    "efficientnetb1",
    "efficientnetb2",
    "efficientnetb3",
    "efficientnetb4",
    "resnet50",
]

In [12]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    create_logger(directory=log_folder, name="logs.txt")
    
for fold_idx in range(5):
    print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")
    
    main(
        fold_idx, 
        log_folder, 
        pretrained_folder + f"pretrained_best_fold{fold_idx}.h5", 
        pretrained_with_contrastive=False
    )
    
    if DEBUG:
        break

Logging results to ../logs/2021-01-31/12/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Loading weights from ../logs/2021-01-31/8/pretrained_best_fold0.h5

 -> Training Model 

Epoch 1/60
Instructions for updating:
Use fn_output_signature instead
30/30 - 10s - loss: 0.2135 - lwlrap: 0.4740 - val_loss: 0.1770 - val_lwlrap: 0.6641
Epoch 2/60
30/30 - 8s - loss: 0.1834 - lwlrap: 0.7522 - val_loss: 0.1476 - val_lwlrap: 0.7174
Epoch 3/60
30/30 - 8s - loss: 0.1559 - lwlrap: 0.7545 - val_loss: 0.1497 - val_lwlrap: 0.7327
Epoch 4/60
30/30 - 8s - loss: 0.1799 - lwlrap: 0.8399 - val_loss: 0.1470 - val_lwlrap: 0.7229
Epoch 5/60
30/30 - 8s - loss: 0.1201 - lwlrap: 0.8712 - val_loss: 0.1264 - val_lwlrap: 0.8138
Epoch 6/60
30/30 - 8s - loss: 0.1058 - lwlrap: 0.8619 - val_loss: 0.1363 - val_lwlrap: 0.6995
Epoch 7/60
30/30 - 8s - loss: 0.0965 - lwlrap: 0.8319 - val_loss: 0.1385 - val_lwlrap: 0.7214
Epoch 8/60
30/30 - 8s - loss: 0.0621 - lwlrap: 0.8491 - val_

Epoch 26/60
30/30 - 8s - loss: 0.0142 - lwlrap: 0.9952 - val_loss: 0.0500 - val_lwlrap: 0.9011
Epoch 27/60
30/30 - 8s - loss: 0.0109 - lwlrap: 0.9904 - val_loss: 0.0586 - val_lwlrap: 0.8956
Epoch 28/60
30/30 - 8s - loss: 0.0316 - lwlrap: 0.9931 - val_loss: 0.0726 - val_lwlrap: 0.8764
Epoch 29/60
30/30 - 8s - loss: 0.0040 - lwlrap: 0.9955 - val_loss: 0.0676 - val_lwlrap: 0.8817
Epoch 30/60
30/30 - 8s - loss: 0.0058 - lwlrap: 0.9947 - val_loss: 0.0581 - val_lwlrap: 0.8806
Epoch 31/60
30/30 - 8s - loss: 0.0020 - lwlrap: 0.9975 - val_loss: 0.0513 - val_lwlrap: 0.8975
Epoch 32/60
30/30 - 8s - loss: 0.0111 - lwlrap: 0.9974 - val_loss: 0.0652 - val_lwlrap: 0.8739
Epoch 33/60
30/30 - 8s - loss: 0.0063 - lwlrap: 0.9935 - val_loss: 0.0712 - val_lwlrap: 0.8627
Epoch 34/60
30/30 - 8s - loss: 0.0012 - lwlrap: 0.9973 - val_loss: 0.0542 - val_lwlrap: 0.8816
Epoch 35/60
30/30 - 8s - loss: 0.0123 - lwlrap: 0.9970 - val_loss: 0.0646 - val_lwlrap: 0.8719
Epoch 36/60
30/30 - 8s - loss: 0.0046 - lwlrap: 0.

Epoch 4/60
30/30 - 8s - loss: 0.1093 - lwlrap: 0.8964 - val_loss: 0.0884 - val_lwlrap: 0.8020
Epoch 5/60
30/30 - 8s - loss: 0.0561 - lwlrap: 0.9113 - val_loss: 0.1079 - val_lwlrap: 0.7903
Epoch 6/60
30/30 - 8s - loss: 0.1618 - lwlrap: 0.8821 - val_loss: 0.1108 - val_lwlrap: 0.7397
Epoch 7/60
30/30 - 8s - loss: 0.0688 - lwlrap: 0.8535 - val_loss: 0.1107 - val_lwlrap: 0.7509
Epoch 8/60
30/30 - 9s - loss: 0.0749 - lwlrap: 0.8864 - val_loss: 0.0597 - val_lwlrap: 0.8478
Epoch 9/60
30/30 - 8s - loss: 0.0533 - lwlrap: 0.9051 - val_loss: 0.0903 - val_lwlrap: 0.7359
Epoch 10/60
30/30 - 8s - loss: 0.1746 - lwlrap: 0.9020 - val_loss: 0.0862 - val_lwlrap: 0.7874
Epoch 11/60
30/30 - 8s - loss: 0.0706 - lwlrap: 0.8789 - val_loss: 0.1039 - val_lwlrap: 0.6926
Epoch 12/60
30/30 - 8s - loss: 0.0457 - lwlrap: 0.9015 - val_loss: 0.0613 - val_lwlrap: 0.8058
Epoch 13/60
30/30 - 8s - loss: 0.0317 - lwlrap: 0.9362 - val_loss: 0.1346 - val_lwlrap: 0.7417
Epoch 14/60
30/30 - 8s - loss: 0.0231 - lwlrap: 0.9711 -